In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.7 MB/s eta 0:00:00


In [3]:
import numpy as np
import nltk
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, GRU, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pickle
from datasets import load_dataset

In [4]:
# Load the Rotten Tomatoes dataset
dataset = load_dataset("rotten_tomatoes")
train_dataset = dataset['train']
valid_dataset = dataset['validation']
test_dataset = dataset['test']

train_text = train_dataset.to_pandas()['text']

max_text_len = 0
for text in train_text:
    max_text_len = max(max_text_len, len(text))

print(max_text_len) #length of longest train sentence
len(train_text) #number of sentences

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.46k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

267


8530

In [7]:
with open('embedding_matrix_300d.pkl', 'rb') as f:
    embedding_matrix = pickle.load(f).astype(np.float32)
    padding = [0 for i in range(300)]
    embedding_matrix = np.insert(embedding_matrix, 0, padding, 0)
    print(type(embedding_matrix))

with open('vocab_word_to_index_300d.pkl', 'rb') as f:
    vocab_word_to_index = pickle.load(f)
    print(vocab_word_to_index) #word index

# Convert to torch tensors
embedding_matrix = tf.convert_to_tensor(embedding_matrix) #adopting the glove embeddings
vocab_size, embedding_dim = embedding_matrix.shape

<class 'numpy.ndarray'>
{'gantzes': 0, 'tatters': 1, 'mischief': 2, 'l': 3, 'kills': 4, 'stamos': 5, 'leveling': 6, 'pretension': 7, 'grade': 8, 'sultry': 9, 'lascivious': 10, 'jez': 11, 'hussein': 12, 'goldman': 13, 'names': 14, 'splat': 15, 'argento': 16, 'news': 17, 'grating': 18, 'probe': 19, 'screenwriter': 20, 'tailor': 21, 'marry': 22, 'blockbusters': 23, 'milks': 24, 'sea': 25, 'eternity': 26, 'salacious': 27, 'incurably': 28, 'mordantly': 29, 'no': 30, 'artists': 31, 'cries': 32, 'created': 33, 'gelati': 34, 'shifty': 35, 'namesake': 36, 'timid': 37, 'miraculously': 38, 'westfeldt': 39, 'heartache': 40, 'plumbs': 41, 'blush': 42, 'renown': 43, 'strung': 44, 'pushes': 45, 'disappoint': 46, 'decides': 47, 'guise': 48, 'qui': 49, 'intrusive': 50, 'sayles': 51, 'sarandon': 52, 'tonight': 53, 'fantastic': 54, 'corners': 55, 'garcía': 56, 'wives': 57, 'frame': 58, 'undertone': 59, 'filmes': 60, 'commodity': 61, 'benigni': 62, 'campaign': 63, 'control': 64, 'named': 65, 'avant': 66, 

In [9]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [10]:
import numpy as np
import tensorflow as tf
import nltk

class SentimentDataset:
    def __init__(self, dataset, word_to_index, max_len=max_text_len):
        self.dataset = dataset
        self.word_to_index = word_to_index
        self.max_len = max_len

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        text = self.dataset[idx]['text']
        label = self.dataset[idx]['label']

        # Tokenization and word-to-index conversion
        text = text.lower()
        word_list = nltk.tokenize.word_tokenize(text)
        word_list = [word.strip("'\"") for word in word_list]  # Remove extra punctuation
        indices = [self.word_to_index.get(word, self.word_to_index.get('<UNK>')) + 1 for word in word_list]
        indices = indices[:self.max_len] + [0] * (self.max_len - len(indices))  # Padding

        return np.array(indices), np.array(label)

    def preprocess_data(self):
        texts = []
        labels = []
        for i in range(len(self.dataset)):
            features, label = self.__getitem__(i)
            texts.append(features)
            labels.append(label)
        return np.array(texts), np.array(labels)

# Create a TensorFlow dataset
def create_tf_dataset(texts, labels, batch_size=32, shuffle=True):
    dataset = tf.data.Dataset.from_tensor_slices((texts, labels))

    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(texts))  # Shuffle the dataset

    dataset = dataset.batch(batch_size)  # Batch the dataset
    return dataset

train_texts, train_labels = SentimentDataset(train_dataset, vocab_word_to_index, max_text_len).preprocess_data()
valid_texts, valid_labels = SentimentDataset(valid_dataset, vocab_word_to_index, max_text_len).preprocess_data()
test_texts, test_labels = SentimentDataset(test_dataset, vocab_word_to_index, max_text_len).preprocess_data()

# Create TensorFlow datasets
train_tf_dataset = create_tf_dataset(train_texts, train_labels, batch_size=32)
valid_tf_dataset = create_tf_dataset(valid_texts, valid_labels, batch_size=32, shuffle = False)
test_tf_dataset = create_tf_dataset(test_texts, test_labels, batch_size=32, shuffle = False)
'''for step, (features, labels) in enumerate(train_tf_dataset):
    print(f"Batch {step + 1}")
    print("Features (inputs):", features.numpy())  # Convert tensor to NumPy for easier reading
    print("Labels (outputs):", labels.numpy())  # Convert tensor to NumPy for easier reading
    print("\n")
    break'''

'for step, (features, labels) in enumerate(train_tf_dataset):\n    print(f"Batch {step + 1}")\n    print("Features (inputs):", features.numpy())  # Convert tensor to NumPy for easier reading\n    print("Labels (outputs):", labels.numpy())  # Convert tensor to NumPy for easier reading\n    print("\n")\n    break'

In [12]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
import random
# Build model
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

def build_model(nb_words, embedding_matrix, max_len):
    model = Sequential(name="Simple_RNN")

    # Embedding layer with pre-trained embeddings
    if embedding_matrix is not None:
        model.add(Embedding(nb_words, embedding_matrix.shape[1], weights=[embedding_matrix], trainable=False))  # Freeze the embedding layer
    else:
        model.add(Embedding(nb_words, 300, trainable=False))

    model.add(SimpleRNN(128,#activation= 'relu',
                        return_sequences=False))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=Adam(learning_rate=0.0001),loss="binary_crossentropy", metrics=['accuracy'])
    return model

nb_words = len(vocab_word_to_index) + 1  # Vocabulary size
model = build_model(nb_words, embedding_matrix, max_text_len)

early_stopping = EarlyStopping(monitor='val_accuracy',patience=5, restore_best_weights=True)
#reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=3, min_lr=0.00001)
train = model.fit(train_tf_dataset, epochs=30, batch_size=32, validation_data=valid_tf_dataset
                  ,callbacks=[early_stopping])


Epoch 1/30
267/267 ━━━━━━━━━━━━━━━━━━━━ 13s 36ms/step - accuracy: 0.4993 - loss: 0.6945 - val_accuracy: 0.5047 - val_loss: 0.6929
Epoch 2/30
267/267 ━━━━━━━━━━━━━━━━━━━━ 17s 31ms/step - accuracy: 0.5383 - loss: 0.6905 - val_accuracy: 0.5516 - val_loss: 0.6883
Epoch 3/30
267/267 ━━━━━━━━━━━━━━━━━━━━ 8s 29ms/step - accuracy: 0.6186 - loss: 0.6593 - val_accuracy: 0.6557 - val_loss: 0.6434
Epoch 4/30
267/267 ━━━━━━━━━━━━━━━━━━━━ 8s 31ms/step - accuracy: 0.6654 - loss: 0.6246 - val_accuracy: 0.6895 - val_loss: 0.6043
Epoch 5/30
267/267 ━━━━━━━━━━━━━━━━━━━━ 10s 31ms/step - accuracy: 0.7253 - loss: 0.5773 - val_accuracy: 0.6979 - val_loss: 0.6157
Epoch 6/30
267/267 ━━━━━━━━━━━━━━━━━━━━ 8s 28ms/step - accuracy: 0.7250 - loss: 0.5755 - val_accuracy: 0.7017 - val_loss: 0.5956
Epoch 7/30
267/267 ━━━━━━━━━━━━━━━━━━━━ 10s 29ms/step - accuracy: 0.7396 - loss: 0.5551 - val_accuracy: 0.7214 - val_loss: 0.5854
Epoch 8/30
267/267 ━━━━━━━━━━━━━━━━━━━━ 11s 31ms/step - accuracy: 0.7459 - loss: 0.5495 - val

In [13]:
print("Simple_RNN Score---> ", model.evaluate(test_tf_dataset))

34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.6749 - loss: 0.6105
Simple_RNN Score--->  [0.5767036080360413, 0.7204502820968628]


In [ ]:
# Get predictions on the test dataset
predictions = model.predict(test_tf_dataset)

# Convert probabilities to binary predictions (0 or 1) using a threshold of 0.5
predicted_labels = (predictions > 0.5).astype(int).flatten()

# Get true labels from the test dataset
true_labels = np.concatenate([y.numpy() for x, y in test_tf_dataset], axis=0)

from sklearn.metrics import confusion_matrix

# Generate confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

print("Confusion Matrix:")
print(conf_matrix)

34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step
Confusion Matrix:
[[226 307]
 [229 304]]


2024-10-24 10:50:55.973759: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
